In [2]:
# Data Preparation Libraries
import pandas as pd
import numpy as np
import datetime

# Data Profiling
import pandas_profiling

# Plot and Visualisations
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import bokeh
%matplotlib inline

# Notebook settings
import warnings
pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', 300)
warnings.filterwarnings('ignore')
import gc

# Modelling libraries
from xgboost.sklearn import XGBClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score

# Data Structure

<img src="https://storage.googleapis.com/kaggle-media/competitions/home-credit/home_credit.png" style="width:1000px;height:600px"/>

### First Glimpse of Data

In [8]:
# train = pd.read_csv(r'C:\Users\c026nedem\MyProjects\Home Credit\previous_application.csv')
train = pd.read_csv('https://drive.google.com/uc?export=download&confirm=1Na3TmqqEluah4T-3hK2Zt1f25b-GbISE')

train.head()

HTTPError: HTTP Error 400: Bad Request

In [9]:
train.NAME_CONTRACT_STATUS.value_counts()

Approved        1036781
Canceled         316319
Refused          290678
Unused offer      26436
Name: NAME_CONTRACT_STATUS, dtype: int64

In [10]:
# Engineer categorical variable by ordinal number replacement

approval_map = {'Approved':1,
               'Canceled':0,
               'Refused':0,
               'Unused offer':0}

train['APPROVAL'] = train.NAME_CONTRACT_STATUS.map(approval_map)
train[['APPROVAL','NAME_CONTRACT_STATUS']].head(10)

,APPROVAL,NAME_CONTRACT_STATUS
0,1,Approved
1,1,Approved
2,1,Approved
3,1,Approved
4,0,Refused
5,1,Approved
6,0,Canceled
7,0,Canceled
8,0,Canceled
9,0,Canceled


In [12]:
train.APPROVAL.value_counts(normalize=True,dropna=False)

1    0.620747
0    0.379253
Name: APPROVAL, dtype: float64

### Tasks
- Inspect the dataset
- Determine numerical variables
- Determine categorical variables
- Determine number of nulls
- Split in train test using random split
- Build a baseline model using only numeric data that predicts APPROVAL field